In [ ]:
# import libraries
import math
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Input
from tensorflow.keras.layers import Reshape, MaxPooling2D
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [ ]:
# import tensorflow libraries
from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, Dense, Flatten

In [ ]:
# import bayesian optimisation libraries
import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.plots import plot_histogram, plot_objective_2D
from skopt.utils import use_named_args
import h5py

In [ ]:
# set hyperparameter tuning upper and lower bounds
dim_learning_rate = Real(low=1e-6, high=1e-2, prior='log-uniform',
                         name='learning_rate')
dim_num_dense_layers = Integer(low=1, high=5, name='num_dense_layers')
dim_num_dense_nodes = Integer(low=5, high=512, name='num_dense_nodes')
dim_num_batch = Integer(low=32, high=512, name='num_batch')
dim_dropout = Categorical(categories=[True, False],name='num_dropout')
dim_batchnorm = Categorical(categories=[True, False], name='num_batchnorm')


In [ ]:
# concatenate all hyperparameters that require tuning
dimensions = [dim_learning_rate,
              dim_num_dense_layers,
              dim_num_dense_nodes,
              dim_num_batch, dim_dropout, dim_batchnorm]


In [ ]:
# use manual hyperparameter selection for default parameters
default_parameters = [1e-5, 1, 16, 64,True,False]


In [ ]:
# define log directory to view models
def log_dir_name(learning_rate, num_dense_layers,
                 num_dense_nodes, num_batch, num_dropout, num_batchnorm):

    # The dir-name for the TensorBoard log-dir.
    s = "./19_logs/lr_{0:.0e}_layers_{1}_nodes_{2}_{3}/"

    # Insert all the hyper-parameters in the dir-name.
    log_dir = s.format(learning_rate,
                       num_dense_layers,
                       num_dense_nodes,
                       num_batch, num_dropout, num_batchnorm)

    return log_dir


In [ ]:
# data preparation
from sklearn.preprocessing import StandardScaler
import pandas as pd

data = pd.read_csv("Data_SABR.txt", header=None, names=[
                   "K", "T", "v", "alpha", "rho", "imp vol"])
targets = data.iloc[:, 5]
data = data.iloc[:, 0:5]
data
scaler = StandardScaler()
data = scaler.fit_transform(data)
train_inputs, test_inputs, train_targets, test_targets = train_test_split(
    data, targets, test_size=0.3)
train_inputs = tf.convert_to_tensor(train_inputs, dtype='float64')
train_targets = tf.convert_to_tensor(train_targets.values, dtype='float64')
test_inputs = tf.convert_to_tensor(test_inputs, dtype='float64')
test_targets = tf.convert_to_tensor(test_targets.values, dtype='float64')
input_size = train_inputs.shape[1]
output_size = 1


In [ ]:
# define early stopping mechanism
LRScheduler = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10,
                                verbose=0, min_delta=0.001, cooldown=10, min_lr=0.00001)


In [ ]:
def create_model(learning_rate, num_dense_layers,
                 num_dense_nodes, num_batch, num_dropout, num_batchnorm):
  
    # Start construction of a Keras Sequential model.
    model = Sequential()

    model.add(InputLayer(input_shape=(input_size,)))


    for i in range(num_dense_layers):
        name = 'layer_dense_{0}'.format(i+1)

        if num_batchnorm:model.add(BatchNormalization())
        if num_dropout:model.add(Dropout(0.1))
        model.add(Dense(num_dense_nodes,
                        activation="relu",
                        name=name))

    model.add(Dense(output_size, activation='linear'))

    # Find the best learning-rate for the Adam method.
    optimizer = Adam(lr=learning_rate)

    # Compile the model so it can be trained.
    model.compile(optimizer="adam", loss='mse', metrics=["mean_absolute_error"])

    return model


In [ ]:
# initialise loss and best model directory
best_accuracy = 0.0


In [ ]:
# bayesian optimisation objective function
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_dense_layers,
            num_dense_nodes, num_batch, num_dropout, num_batchnorm):

    # Print the hyper-parameters.
    print('learning rate: {0:.1e}'.format(learning_rate))
    print('num_dense_layers:', num_dense_layers)
    print('num_dense_nodes:', num_dense_nodes)
    print('num_batch:', num_batch)
    print('num_dropout:', num_dropout)
    print('num_batchnorm:', num_batchnorm)

    # Create the neural network with these hyper-parameters.
    model = create_model(learning_rate=learning_rate,
                         num_dense_layers=num_dense_layers,
                         num_dense_nodes=num_dense_nodes,
                         num_batch=num_batch, num_dropout=num_dropout, num_batchnorm=num_batchnorm)

    # Dir-name for the TensorBoard log-files.
    log_dir = log_dir_name(learning_rate, num_dense_layers,
                           num_dense_nodes, num_batch, num_dropout, num_batchnorm)

    callback_log = TensorBoard(
        log_dir=log_dir,
        histogram_freq=0,
        write_graph=True,
        write_grads=False,
        write_images=False)

    # Use Keras to train the model.
    history=model.fit(train_inputs, 
          train_targets, 
          batch_size=num_batch,  
          epochs=250,
          validation_split = 0.1,
           callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, verbose=False, min_delta=0.000001),
                          LRScheduler],
          verbose=True)

    # Get the loss on the validation-set
    accuracy = history.history['val_loss'][-1]

    # Print the loss
    print("Loss: {0}".format(accuracy))

    # Save the model if it improves on the best-found performance.
    global best_accuracy

    # If the loss of the saved model is improved 
    if accuracy > best_accuracy:
        # Save the new model 
        model.save('models/best_model.h5')

        best_accuracy = accuracy

    # Delete the Keras model with these hyper-parameters from memory.
    del model


    K.clear_session()

    return accuracy


In [ ]:
# test function is working
fitness(x=default_parameters)


In [ ]:
# Begin bayesian optimization
search_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            n_calls=40,
                            x0=default_parameters)
